In [1]:
# Importing necessary libraries
import cv2
import numpy as np
from tensorflow.keras.layers import Dense, Input, Dropout, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt

# Define a function to create the eye state detection model
def eye_state_detection_model(input_shape):
    # Define the CNN architecture
    inputs = Input(input_shape)
    x = Conv2D(32, (3, 3), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)
    
    x = Conv2D(64, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)
    
    x = Flatten()(x)
    x = Dense(128)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)
    
    outputs = Dense(1, activation='sigmoid')(x)  # Output layer with sigmoid activation for binary classification
    
    # Create the model
    model = Model(inputs, outputs)
    
    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# Define input image dimensions
img_size = 48
input_shape = (img_size, img_size, 1)

# Initialize the eye state detection model
eye_state_model = eye_state_detection_model(input_shape)
eye_state_model.summary()

# Define data directories for real-time data capture (e.g., webcam feed)
# For demonstration purposes, let's assume you're using OpenCV for video capture

# Initialize the video capture device
cap = cv2.VideoCapture(0)  # Use '0' for the default webcam, or provide the device index

# Loop for real-time detection
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Preprocess the frame (resize, normalize, etc.)
    resized_frame = cv2.resize(gray, (img_size, img_size))
    normalized_frame = resized_frame / 255.0  # Normalize pixel values to [0, 1]
    input_frame = np.expand_dims(normalized_frame, axis=-1)  # Add batch dimension

    # Perform eye state prediction
    eye_state_prediction = eye_state_model.predict(np.array([input_frame]))
    
    # Determine the predicted eye state (open or closed) based on the prediction threshold
    predicted_eye_state = "Open" if eye_state_prediction > 0.5 else "Closed"
    
    # Display the frame with predicted eye state
    cv2.putText(frame, predicted_eye_state, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    cv2.imshow('Eye State Detection', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture device and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 48, 48, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 48, 48, 32)        320       
                                                                 
 batch_normalization (BatchN  (None, 48, 48, 32)       128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 48, 48, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 24, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 24, 24, 32)        0     